In [1]:
import json
import os
import pandas as pd
import re
import requests

In [2]:
parl_api_url = 'https://lop.parl.ca/ParlinfoWebAPI'
accept_json = {'Accept': 'application/json'}

In [199]:
def drop_time(df):
    date_cols = [col for col in df.columns if col.endswith('Date')]
    for col in date_cols:
        df[col] = df[col].str[:10]

def drop_french(df):
    to_drop = [col for col in df.columns if col.endswith('Fr')]
    df.drop(columns=to_drop, inplace=True)
    
def drop_empty_cols(df):
    to_drop = [col for col in df.columns if df[col].count() == 0]
    df.drop(columns=to_drop, inplace=True)

def drop_unsupported_cols(df):
    df.drop(columns=['Documents', 'Senator'], inplace=True, errors='ignore')
    
def cleanup(df):
    drop_time(df)
    drop_french(df)
    drop_empty_cols(df)
    drop_unsupported_cols(df)

In [4]:
r = requests.get(parl_api_url + '/Person/SearchAndRefine?refiners=4-1,', headers=accept_json)
d = r.json()
len(d)

430

In [ ]:
r = requests.get(parl_api_url + '/Person/SearchAndRefine?refiners=28-1,28-2,28-3,', headers=accept_json)
d = r.json()
len(d)

In [5]:
len(d[0]['Roles'])

2

In [6]:
rows = [{k: (p[k] if k in p else None) for k in ['PersonId', 'LastName', 'UsedFirstName', 'ProvincialExperienceEN', 'MunicipalExperienceEn']} for p in d]
len(rows)

430

In [7]:
df = pd.DataFrame(rows)
df

,PersonId,LastName,UsedFirstName,ProvincialExperienceEN,MunicipalExperienceEn
0,18421,Aboultaif,Ziad,None,
1,20137,Aitchison,Scott,None,
2,17852,Albas,Dan,None,
3,8947,Alghabra,Omar,None,
4,18479,Alleslev,Leona,None,
...,...,...,...,...,...
425,18279,Yurdiga,David,None,
426,18535,Zahid,Salma,None,
427,20099,Zann,Lenore,None,
428,17951,Zimmer,Bob,None,


In [8]:
df['ProvincialExperienceEN'].value_counts()

Series([], Name: ProvincialExperienceEN, dtype: int64)

In [ ]:
df[df['LastName'] == 'Aglukkaq']

In [ ]:
r = requests.get(parl_api_url + '/Person/GetPersonWebProfile/4487', headers=accept_json)
d = r.json()
len(d)

In [ ]:
sorted(d.keys())

In [ ]:
for k in list(d.keys()):
    if k.endswith('Fr'):
        del d[k]

In [ ]:
person = d['Person']
name = f"{person['UsedFirstName']} {person['LastName']}"
name = person['DisplayName']
name

In [ ]:
filename = name.replace(' ', '_') + '.json'
filename = name + '.json'
filename

In [ ]:
with open(filename, 'w') as f:
    json.dump(d, f, indent=2)

In [ ]:
!code $filename

In [ ]:
sorted(d.keys())

In [ ]:
sorted(d['Person'].keys())

In [ ]:
[key for key in sorted(d['Person']['Roles'][0].keys()) if not key.endswith('Fr')]

In [18]:
def num_prefix(str):
    return int(str.split('-')[0])

def name_suffix(str):
    return str.split('-')[-1]

In [34]:
# Match people .json files
def person_files():
    people_dir = '../data/people'
    pattern = re.compile('[0-9]+-.+\.json$')
    matching_files = filter(lambda d: pattern.match(d), os.listdir(people_dir))
    return sorted(matching_files, key=name_suffix)

len(person_files())

5180

In [22]:
def person_recs(n = None):
    files = person_files()
    for file in files[:n] if n else files:
        with open(os.path.join(people_dir, file)) as f:
            yield json.load(f)

In [25]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
role_cols = ['PersonRoleId', 'ParliamentNumber', 'PartyEn', 'ToBeStyledAsEn',
             'OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 
             'IsMP', 'IsSenator', 'IsActing',
             'StartDate', 'StartDateIsApproximate', 
             'EndDate', 'EndDateIsApproximate', 'EndReasonEn', 'EndReasonTypeEn', 
             'NotesEn']

rows = []
for rec in person_recs():
    person = rec['Person']
    person_tuple = tuple(person[col] for col in person_cols)
    for role in rec['FederalExperience']:
        role_tuple = tuple(role[col] for col in role_cols)
        row = person_tuple + role_tuple
        rows.append(row)
len(rows)

53900

In [26]:
df = pd.DataFrame(rows, columns = person_cols + role_cols) \
  .sort_values(['LastName', 'UsedFirstName', 'PersonId', 'StartDate', 'PersonRoleId'])
df = df.drop_duplicates()
len(df)

53781

In [27]:
cleanup(df)
df.sample(5)

,PersonId,LastName,UsedFirstName,PersonRoleId,ParliamentNumber,PartyEn,ToBeStyledAsEn,OrganizationTypeEn,OrganizationLongEn,GroupingTitleEn,...,IsMP,IsSenator,IsActing,StartDate,StartDateIsApproximate,EndDate,EndDateIsApproximate,EndReasonEn,EndReasonTypeEn,NotesEn
18334,18477,Fraser,Colin,265633,42,Liberal Party of Canada,,Party,Liberal Party of Canada,Parliamentarian,...,False,False,False,2015-10-19,False,2019-10-20,False,None,None,None
43443,7917,Rideout,Margaret Isabel,74903,27,Liberal Party of Canada,,Constituency,Westmorland,Parliamentarian,...,True,False,False,1964-11-09,False,1968-06-24,False,None,None,None
53300,16783,Wrzesnewskyj,Borys,312662,39,Liberal Party of Canada,None,Government,Foreign Affairs and International Trade,House of Commons Roles,...,False,False,False,2006-02-23,False,2006-08-23,False,None,None,None
51598,7937,Walker,David James,103325,23,Progressive Conservative Party,Parliamentary Assistant to the Minister of Jus...,Government,Justice,House of Commons Roles,...,False,False,False,1957-08-19,False,1958-02-01,False,None,None,None
20783,14023,Gray,Herbert Eser (Herb),139716,32,Liberal Party of Canada,President of the Treasury Board,Government,Treasury Board,Cabinet,...,False,False,False,1982-09-30,False,1984-09-16,False,None,None,None


In [ ]:
df.loc[39471]

In [28]:
df['ToBeStyledAsEn'].value_counts()[:30]

                                                                                      43865
Minister Without Portfolio                                                              113
Minister of Justice and Attorney General of Canada                                       87
Secretary of State of Canada                                                             75
Minister of Public Works                                                                 69
Postmaster General                                                                       69
Minister of Labour                                                                       59
Solicitor General of Canada                                                              57
Minister of Agriculture                                                                  56
Minister of National Defence                                                             56
President of the Privy Council                                                  

In [29]:
filename = os.path.join(people_dir, 'fed_roles.csv')
df.to_csv(filename, index=False, encoding='utf8')

In [ ]:
!open $filename

In [206]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
education_cols = ['SchoolNameLongEn', 'FieldOfStudyEn', 'DiplomaLongEn', 'GraduationYear']

rows = []
for rec in person_recs():
    person = rec['Person']
    person_tuple = tuple(person[col] for col in person_cols)
    if person['Education']:
        for edu in person['Education']:
            edu_tuple = tuple(edu[col] for col in education_cols)
            row = person_tuple + edu_tuple
            rows.append(row)
len(rows)


5711

In [207]:
df = pd.DataFrame(rows, columns = person_cols + education_cols) \
  .sort_values(['LastName', 'UsedFirstName', 'PersonId', 'GraduationYear'])
df = df.drop_duplicates()
len(df)

5708

In [208]:
df.sample(25)

,PersonId,LastName,UsedFirstName,SchoolNameLongEn,FieldOfStudyEn,DiplomaLongEn,GraduationYear
5684,2578,de Corneille,Roland,Trinity College,Theology,Licentiate,1953.0
4926,13186,Skoke,Roseanne,Dalhousie University,Arts,Bachelor,NaN
2872,20147,Lattanzio,Patricia,McGill University,None,Bachelor,1985.0
4961,18483,Erskine-Smith,Nathaniel,University of Oxford,Constitutional Law and Political Science,Master,NaN
752,2885,Bégin,Monique,McGill University,Unknown,Doctorate,1965.0
2343,13892,Holmes,John Robert,University of Western Ontario,Medicine,Doctorate,NaN
748,10793,Byrne,Gerry,Dalhousie University,Biology,Bachelor,1991.0
619,18596,Breton,Pierre,University of Quebec at Montréal,Business Administration,Bachelor,NaN
2706,8138,Knutson,Gar,University of Windsor,Laws,Bachelor,1981.0
4197,14372,Pearson,Landon,University of Ottawa,Education,Master,1978.0


In [209]:
filename = os.path.join(people_dir, 'education.csv')
df.to_csv(filename, index=False, encoding='utf8')

In [ ]:
education_cols

In [ ]:
parl_num = 42
parl = df[df['ParliamentNumber'] == parl_num]
parl

In [ ]:
filename = os.path.join(people_dir, f'parl_{parl_num}_roles.csv')
parl42.to_csv(filename, index=False, encoding='utf8')

In [ ]:
!open $filename

In [ ]:
mp_roles = df[df['IsMP']].drop_duplicates()
len(mp_roles)

In [ ]:
mp_roles.nunique().sort_values(ascending=False)

In [ ]:
mp_roles.fillna('').groupby(['GroupingTitleEn', 'OrganizationTypeEn', 'PortFolioEn', 'NameEn'])[['PersonRoleId']].count()

In [ ]:
mp_roles.fillna('').groupby(['GroupingTitleEn', 'OrganizationTypeEn']).nunique()

In [ ]:
mp_roles.nunique()

In [ ]:
mp_roles = mp_roles.drop(['GroupingTitleEn', 'PortFolioEn', 'NameEn', 'IsMP', 'IsSenator', 'IsActing'], axis=1)
mp_roles

In [ ]:
mp_roles['PersonRoleId'].value_counts()[:25]

In [ ]:
cols = ['PersonId', 'LastName', 'UsedFirstName', 'PersonRoleId', 'StartDate', 'EndDate',
        'PartyEn', 'OrganizationTypeEn', 'OrganizationLongEn']
mp_roles = mp_roles[cols]
mp_roles

In [ ]:
mp_roles.to_csv(os.path.join(people_dir, 'mp_roles.csv'), index=False, encoding='utf8')

In [ ]:
df.columns

In [ ]:
df['GroupingTitleEn'].fillna('').value_counts()

In [ ]:
df['OrganizationTypeEn'].fillna('').value_counts()

In [ ]:
df['IsMP'].fillna('').value_counts()

In [ ]:
parl_roles = df[df['GroupingTitleEn'] == 'Parliamentarian']
parl_roles.count()

In [ ]:
mps_df = df[df['IsMP']]
mps_df[['GroupingTitleEn', 'OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 'IsMP']].nunique()

In [ ]:
mps_df['GroupingTitleEn'].value_counts()

In [ ]:
mps_df['OrganizationTypeEn'].value_counts()

In [ ]:
mps_df['OrganizationLongEn'].value_counts()

In [ ]:
mps_df['PortFolioEn'].value_counts()

In [ ]:
mps_df['NameEn'].value_counts()

In [ ]:
parl_roles.fillna('') \
  .groupby(['OrganizationTypeEn', 'OrganizationLongEn', 'GroupingTitleEn', 'PortFolioEn', 'NameEn', 'IsMP'])[['PersonRoleId']].count() \
  [:25]

In [ ]:
df['PartyEn'].value_counts()

In [ ]:
df['IsActing'].value_counts()

In [ ]:
df.groupby(['EndReasonTypeEn', 'EndReasonEn'])[['PersonRoleId']].count()

In [ ]:
df[df['EndReasonTypeEn'] == 'Resignation']

In [ ]:
df2 = df[~df['EndReasonTypeEn'].isna() | ~df['EndReasonEn'].isna()]
df2

In [ ]:
df[['EndReasonEn', 'EndReasonTypeEn']].drop_duplicates()

In [ ]:
df.fillna('na').pivot_table(
    index=['EndReasonTypeEn', 'EndReasonEn'],
    values=['PartyEn', 'PersonId', 'PersonRoleId'],
    aggfunc='nunique'
)[['PartyEn', 'PersonId', 'PersonRoleId']]

In [ ]:
df.fillna('na').groupby(['EndReasonTypeEn', 'EndReasonEn'])[['PartyEn', 'PersonId', 'PersonRoleId']].nunique() \
  .sort_values('PersonRoleId', ascending=False)

In [ ]:
df[df['EndReasonEn'] == 'Resigned between May 30, 1930 and September 9,1930.']

In [ ]:
row = df.loc[107319]
row

In [ ]:
row = 107319
df.loc[row, 'NotesEn'] = df.loc[row, 'EndReasonEn']
df.loc[row, 'EndReasonEn'] = None

In [ ]:
df['PersonRoleId'].count(), df['PersonRoleId'].nunique()

In [ ]:
df[:50]

In [ ]:
df[:50].sort_values(['LastName', 'UsedFirstName', 'StartDate', 'PersonRoleId'])

In [ ]:
len(df)

In [ ]:
path = os.path.join(people_dir, 'person_roles.csv')
df.to_csv(path, index=False, encoding='utf8')

In [200]:
def extract_roles(role_type):
    global rows
    global df
    
    person_cols = ['PersonId', 'LastName', 'UsedFirstName']

    rows = []
    for rec in person_recs():
        person = {col: rec['Person'][col] for col in person_cols}
        roles = rec[role_type]
        if roles:
            for role in roles:
                classes = role['Classes']
                if classes is not None:
                    class_names = [c['RoleClassNameEn'] for c in classes]
                    if None in class_names:
                        print(person, "class names:", class_names)
                    role['Classes'] = '|'.join(filter(None, class_names))

                mp_info = role['MemberOfParliament']
                if mp_info is not None:
                    role['MemberOfParliament'] = mp_info['OccupationTypeEn']
                    
                row = {**person, **role}
                rows.append(row)
                
    df = pd.DataFrame(rows).set_index('PersonRoleId')
    df = df.sort_values(['LastName', 'UsedFirstName', 'PersonId', 'StartDate', 'RoleId'])
    cleanup(df);
    df = df.drop_duplicates()
    return df

In [152]:
df = extract_roles('ProvincialExperience')
path = os.path.join(people_dir, 'provincial_experience.csv')
df.to_csv(path, encoding='utf8')
df.count()

PersonId                  5419
LastName                  5419
UsedFirstName             5419
StartDate                 5395
StartDateIsApproximate    5419
EndDate                   5313
EndDateIsApproximate      5419
EndReasonTypeEn              3
SourceOfInformationEn        8
IsActing                  5419
HasCrossedTheFloor        5419
OrganizationId            5419
OrganizationLongEn        5419
OrganizationShortEn       5388
OrganizationAcronymEn     5316
OrganizationTypeId        5419
OrganizationTypeEn        5419
OrganizationProvinceEn       2
OrganizationHasProfile    5419
IsSenatorialDivision      5419
ToBeStyledAsEn            5313
GroupId                   5419
GroupingTitleEn           5419
GroupingOrder             5419
ParliamentStart            265
ParliamentEnd              265
ParliamentNumber          5419
PartyOrganizationId       5419
PartyEn                   5145
PartyStartDate            5145
PartyEndDate              5031
PersonPersonId            5419
MemberOf

In [153]:
df['Classes'].drop_duplicates()

PersonRoleId
10292                                                  Prov
99321     Minister|Provincial and Territorial Responsabi...
62707                                       Parliamentarian
116461    Federal Officers and Executives|Minister|Parli...
41929     Political Party|Federal Officers and Executive...
140062         Federal Officers and Executives|Party Leader
56032                       Minister|Parliamentary function
104996                      Federal Officers and Executives
119106    Federal Officers and Executives|Minister|Provi...
71745     Salaries in the House of Commons|Minister|Parl...
132670    Minister|Parliamentary function|Provincial and...
111815    Salaries in the House of Commons|Federal Offic...
137362    Parliamentary function|Provincial and Territor...
76553                  Prov|Minister|Parliamentary function
151809    Minister|Provincial and Territorial Responsabi...
57219     SenatorCabinet|Minister|Parliamentary function...
2156                       

In [154]:
df

,PersonId,LastName,UsedFirstName,StartDate,StartDateIsApproximate,EndDate,EndDateIsApproximate,EndReasonTypeEn,SourceOfInformationEn,IsActing,...,PersonPersonId,MemberOfParliament,IsMP,IsSenator,NotesEn,IsCurrent,RoleId,NameEn,Ordinal,Classes
PersonRoleId,,,,,,,,,,,,,,,,,,,,,
10292,8200,Abbott,John Joseph Caldwell,1857-01-01,True,1867-01-01,True,None,None,False,...,0,None,False,False,None,False,2997,Provincial Constituency Member,NaN,Prov
99321,14709,Achim,Honoré,1917-12-15,False,1921-10-13,False,None,None,False,...,0,None,False,False,None,False,720,Elected Representative,0.0,Minister|Provincial and Territorial Responsabi...
62707,14709,Achim,Honoré,1917-12-15,False,1921-10-13,False,None,None,False,...,0,None,False,False,None,False,722,Party Member,NaN,Parliamentarian
88656,14709,Achim,Honoré,1917-12-15,False,1921-10-13,False,None,None,False,...,0,None,False,False,None,False,2997,Provincial Constituency Member,NaN,Prov
69657,1331,Adams,Michael,1870-06-01,True,1874-06-01,True,None,None,False,...,0,None,False,False,None,False,720,Elected Representative,0.0,Minister|Provincial and Territorial Responsabi...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6845,2773,Évanturel,Gustave,1911-12-11,False,1923-06-24,False,None,None,False,...,0,None,False,False,None,False,2997,Provincial Constituency Member,NaN,Prov
6845,2773,Évanturel,Gustave,1911-12-11,False,1923-06-24,False,None,None,False,...,0,None,False,False,None,False,2997,Provincial Constituency Member,NaN,Prov
6845,2773,Évanturel,Gustave,1911-12-11,False,1923-06-24,False,None,None,False,...,0,None,False,False,None,False,2997,Provincial Constituency Member,NaN,Prov


In [196]:
df = extract_roles('FederalExperience')
path = os.path.join(people_dir, 'federal_experience.csv')
df.to_csv(path, encoding='utf8')
df.count()

{'PersonId': 20049, 'LastName': 'Francis', 'UsedFirstName': 'Brian'} class names: ['Caucus Chair', 'Officers and Officials of Parliament', 'Parliamentarian Federal Role Senate', 'Political Officers', 'Political Party', None]
{'PersonId': 12245, 'LastName': 'Hubley', 'UsedFirstName': 'Elizabeth'} class names: ['Whip', 'Parliamentarian Federal Role Senate', None, 'Senate Roles']
{'PersonId': 14247, 'LastName': 'Merchant', 'UsedFirstName': 'Pana'} class names: ['Whip', 'Parliamentarian Federal Role Senate', None, 'Senate Roles']


PersonId                  53894
LastName                  53894
UsedFirstName             53894
StartDate                 53891
StartDateIsApproximate    53894
EndDate                   50890
EndDateIsApproximate      53894
EndReasonEn                1148
EndReasonTypeEn            6447
SourceOfInformationEn      2687
IsActing                  53894
ActingTextEn                 20
HasCrossedTheFloor        53894
OrganizationId            53894
OrganizationLongEn        53894
OrganizationShortEn       46790
OrganizationAcronymEn     46951
OrganizationTypeId        53894
OrganizationTypeEn        53894
OrganizationProvinceEn    17476
OrganizationHasProfile    53894
IsSenatorialDivision      53894
ToBeStyledAsEn            48150
GroupId                   53894
PortFolioEn                8595
GroupingTitleEn           53894
GroupingOrder             53894
ParliamentStart           53673
ParliamentEnd             52039
ParliamentNumber          53894
PartyOrganizationId       53894
PartyEn 

In [201]:
df = extract_roles('MunicipalExperience')
path = os.path.join(people_dir, 'municipal_experience.csv')
df.to_csv(path, encoding='utf8')
df.count()

PersonId                  2887
LastName                  2887
UsedFirstName             2887
StartDate                 2038
StartDateIsApproximate    2887
EndDate                   2041
EndDateIsApproximate      2887
IsActing                  2887
HasCrossedTheFloor        2887
OrganizationId            2887
OrganizationLongEn        2887
OrganizationShortEn       2785
OrganizationAcronymEn     2785
OrganizationTypeId        2887
OrganizationTypeEn        2887
OrganizationHasProfile    2887
IsSenatorialDivision      2887
ToBeStyledAsEn            2811
GroupId                   2887
GroupingTitleEn             33
GroupingOrder             2887
ParliamentStart              5
ParliamentEnd                5
ParliamentNumber          2887
PartyOrganizationId       2887
PartyEn                      5
PartyStartDate               5
PartyEndDate                 5
PersonPersonId            2887
IsMP                      2887
IsSenator                 2887
NotesEn                    316
IsCurren

In [203]:
df = extract_roles('MilitaryExperience')
path = os.path.join(people_dir, 'military_experience.csv')
df.to_csv(path, encoding='utf8')
df.count()

PersonId                  1931
LastName                  1931
UsedFirstName             1931
StartDate                 1011
StartDateIsApproximate    1931
EndDate                    930
EndDateIsApproximate      1931
IsActing                  1931
HasCrossedTheFloor        1931
OrganizationId            1931
OrganizationLongEn        1931
OrganizationShortEn       1923
OrganizationAcronymEn     1924
OrganizationTypeId        1931
OrganizationTypeEn        1931
OrganizationHasProfile    1931
IsSenatorialDivision      1931
ToBeStyledAsEn            1923
GroupId                   1931
GroupingTitleEn              2
GroupingOrder             1931
ParliamentNumber          1931
PartyOrganizationId       1931
PersonPersonId            1931
IsMP                      1931
IsSenator                 1931
NotesEn                    513
IsCurrent                 1931
RoleId                    1931
NameEn                    1931
Ordinal                    141
Classes                   1930
dtype: i

In [205]:
df = extract_roles('Education')
path = os.path.join(people_dir, 'education.csv')
df.to_csv(path, encoding='utf8')
df.count()

KeyError: 'Education'

In [202]:
df

,PersonId,LastName,UsedFirstName,StartDate,StartDateIsApproximate,EndDate,EndDateIsApproximate,IsActing,HasCrossedTheFloor,OrganizationId,...,PartyEndDate,PersonPersonId,IsMP,IsSenator,NotesEn,IsCurrent,RoleId,NameEn,Ordinal,Classes
PersonRoleId,,,,,,,,,,,,,,,,,,,,,
5181,8200,Abbott,John Joseph Caldwell,1887-01-01,True,1888-01-01,True,False,False,5470,...,None,0,False,False,None,False,223,Mayor,NaN,Municipal
163336,17878,Adams,Eve,2003-01-01,True,2011-05-01,True,False,False,5388,...,None,0,False,False,None,False,1227,Councillor,NaN,Municipal
140054,14267,Adams,Peter,1970-01-01,True,1972-01-01,True,False,False,6506,...,None,0,False,False,None,False,226,Member,300.0,Committee|ElectionProcessCloseDissolution
112502,15802,Adams,Willie,1965-01-01,True,1972-01-01,True,False,False,7037,...,None,0,False,False,Settlement Council,False,1227,Councillor,NaN,Municipal
88748,15802,Adams,Willie,1972-01-01,True,1976-01-01,True,False,False,7037,...,None,0,False,False,Hamlet Council,False,1227,Councillor,NaN,Municipal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51608,7778,Zaplitny,Frederick Samuel,1950-01-01,True,1952-01-01,True,False,False,2370,...,None,0,False,False,None,False,1227,Councillor,NaN,Municipal
281211,17297,Zarac,Lise,2005-01-01,True,2008-10-01,True,False,False,5470,...,None,0,False,False,None,False,1227,Councillor,NaN,Municipal
51912,5305,de Savoye,Pierre,1976-01-01,True,1977-01-01,True,False,False,7612,...,None,0,False,False,None,False,731,Municipal Councillor,NaN,Municipal


In [189]:
{col:list(df[col].map(type).drop_duplicates()) for col in df.columns}

{'PersonId': [int],
 'LastName': [str],
 'UsedFirstName': [str],
 'StartDate': [str, NoneType],
 'StartDateIsApproximate': [bool],
 'EndDate': [str, NoneType],
 'EndDateIsApproximate': [bool],
 'EndReasonEn': [NoneType, str],
 'EndReasonTypeEn': [NoneType, str],
 'SourceOfInformationEn': [NoneType, str],
 'IsActing': [bool],
 'ActingTextEn': [NoneType, str],
 'HasCrossedTheFloor': [bool],
 'OrganizationId': [int],
 'OrganizationLongEn': [str],
 'OrganizationShortEn': [str, NoneType],
 'OrganizationAcronymEn': [str, NoneType],
 'OrganizationTypeId': [int],
 'OrganizationTypeEn': [str],
 'OrganizationProvinceEn': [str, NoneType],
 'OrganizationHasProfile': [bool],
 'IsSenatorialDivision': [bool],
 'ToBeStyledAsEn': [str, NoneType],
 'GroupId': [int],
 'PortFolioEn': [NoneType, str],
 'GroupingTitleEn': [str],
 'GroupingOrder': [int],
 'ParliamentStart': [str, NoneType],
 'ParliamentEnd': [str, NoneType],
 'ParliamentNumber': [int],
 'PartyOrganizationId': [int],
 'PartyEn': [str, NoneTyp

In [192]:
len(rows)

53900

In [193]:
rows[-1]

{'PersonId': 2773,
 'LastName': 'Évanturel',
 'UsedFirstName': 'Gustave',
 'PersonRoleId': 81495,
 'NameFr': 'Membre du parti',
 'StartDate': '1925-10-29T00:00:00Z',
 'StartDateIsApproximate': False,
 'EndDate': '1926-09-13T00:00:00Z',
 'EndDateIsApproximate': False,
 'EndReasonEn': None,
 'EndReasonFr': None,
 'EndReasonTypeEn': None,
 'EndReasonTypeFr': None,
 'SourceOfInformationEn': None,
 'SourceOfInformationFr': None,
 'IsActing': False,
 'ActingTextEn': None,
 'ActingTextFr': None,
 'HasCrossedTheFloor': False,
 'OrganizationId': 4831,
 'OrganizationLongEn': 'Liberal Party of Canada',
 'OrganizationLongFr': 'Parti libéral du Canada',
 'OrganizationShortEn': 'Liberal',
 'OrganizationShortFr': 'Libéral',
 'OrganizationAcronymEn': 'Lib.',
 'OrganizationAcronymFr': 'Lib.',
 'OrganizationTypeId': 4,
 'OrganizationTypeEn': 'Party',
 'OrganizationTypeFr': 'Parti',
 'OrganizationProvinceEn': None,
 'OrganizationProvinceFr': None,
 'OrganizationHasProfile': True,
 'IsSenatorialDivision':

In [ ]:
df.count()

# Candidates
From https://lop.parl.ca/sites/ParlInfo/default/en_CA/ElectionsRidings/Elections

In [210]:
r = requests.get(parl_api_url + '/Parliament/GetCandidates', headers=accept_json)
d = r.json()
len(d)

44443

In [211]:
candidates = pd.DataFrame(d)
candidates.count()

ParliamentNumber              44443
ElectionId                    44443
IsGeneral                     44443
ElectionDate                  44443
ConstituencyId                44443
ConstituencyEn                44443
ConstituencyFr                44443
ProvinceEn                    44443
ProvinceFr                    44443
DisplayName                   44443
ElectionCandidateId           44443
ElectionProcessCandidateId    44443
PersonId                      44032
PersonLastFirstName           16533
PersonRoleId                      0
ElectionCanadaFirstName       44232
ElectionCanadaLastName        44396
ElectionCanadaMiddleName        305
Gender                        44427
OccupationEn                  42100
OccupationFr                  42098
PartyOrganizationId           44443
PartyNameEn                   44443
PartyNameFr                   44443
Votes                         44443
HasProfile                    44443
ResultLongEn                  44443
ResultLongFr                

In [212]:
cleanup(candidates)
candidates.count()

ParliamentNumber              44443
ElectionId                    44443
IsGeneral                     44443
ElectionDate                  44443
ConstituencyId                44443
ConstituencyEn                44443
ProvinceEn                    44443
DisplayName                   44443
ElectionCandidateId           44443
ElectionProcessCandidateId    44443
PersonId                      44032
PersonLastFirstName           16533
ElectionCanadaFirstName       44232
ElectionCanadaLastName        44396
ElectionCanadaMiddleName        305
Gender                        44427
OccupationEn                  42100
PartyOrganizationId           44443
PartyNameEn                   44443
Votes                         44443
HasProfile                    44443
ResultLongEn                  44443
OtherResultLongEn               613
dtype: int64

In [214]:
candidates = candidates.sort_values(['ParliamentNumber', 'ProvinceEn', 'ConstituencyEn', 'Votes', 'DisplayName'])
candidates[-20:]

,ParliamentNumber,ElectionId,IsGeneral,ElectionDate,ConstituencyId,ConstituencyEn,ProvinceEn,DisplayName,ElectionCandidateId,ElectionProcessCandidateId,...,ElectionCanadaLastName,ElectionCanadaMiddleName,Gender,OccupationEn,PartyOrganizationId,PartyNameEn,Votes,HasProfile,ResultLongEn,OtherResultLongEn
44408,43,802,True,2019-10-21,11632,Saskatoon--University,Saskatchewan,"Norris, Jan",45689,14393,...,Norris,None,M,Artist,3388,Green Party of Canada,1401,True,Defeated,None
44407,43,802,True,2019-10-21,11632,Saskatoon--University,Saskatchewan,"Hayton, Susan",45688,14392,...,Hayton,None,F,Physician,4831,Liberal Party of Canada,6146,True,Defeated,None
44406,43,802,True,2019-10-21,11632,Saskatoon--University,Saskatchewan,"Card, Claire",45687,14391,...,Card,None,F,Professor,5774,New Democratic Party,13994,True,Defeated,None
44410,43,802,True,2019-10-21,11632,Saskatoon--University,Saskatchewan,"Tochor, Corey",45691,14395,...,Tochor,None,M,Entrepreneur,2159,Conservative Party of Canada,24514,True,Elected,None
44416,43,802,True,2019-10-21,8024,Souris--Moose Mountain,Saskatchewan,"Patron, Travis",45697,14491,...,Patron,None,M,CEO,15296,Canadian Nationalist Party,168,True,Defeated,None
44415,43,802,True,2019-10-21,8024,Souris--Moose Mountain,Saskatchewan,"Mergel, Judy",45696,14490,...,Mergel,None,F,Tai Chi Instructor,3388,Green Party of Canada,681,True,Defeated,None
44417,43,802,True,2019-10-21,8024,Souris--Moose Mountain,Saskatchewan,"Zajac, Phillip",45698,14492,...,Zajac,None,M,Mortgage Specialist,15161,People's Party of Canada,702,True,Defeated,None
44412,43,802,True,2019-10-21,8024,Souris--Moose Mountain,Saskatchewan,"Ames-Sinclair, Javin",45693,14487,...,Ames-Sinclair,None,M,Student,4831,Liberal Party of Canada,1718,True,Defeated,None
44413,43,802,True,2019-10-21,8024,Souris--Moose Mountain,Saskatchewan,"Hicks, Ashlee",45694,14488,...,Hicks,None,F,Retail Worker,5774,New Democratic Party,3214,True,Defeated,None
44414,43,802,True,2019-10-21,8024,Souris--Moose Mountain,Saskatchewan,"Kitchen, Robert Gordon",45695,14489,...,Kitchen,None,M,Chiropractor,2159,Conservative Party of Canada,35067,True,Elected,None


In [215]:
# Constituencies with ties
wins = candidates[candidates['ResultLongEn'] == 'Elected']
cols = ['ParliamentNumber', 'ElectionId', 'IsGeneral', 'ElectionDate', 'ProvinceEn', 'ConstituencyId', 'ConstituencyEn', 'Votes', 'ResultLongEn']
grp = wins.groupby(cols, as_index=False)[['DisplayName']].count()
ties = grp[grp['DisplayName'] > 1]
ties.sort_values('Votes', ascending=False)

,ParliamentNumber,ElectionId,IsGeneral,ElectionDate,ProvinceEn,ConstituencyId,ConstituencyEn,Votes,ResultLongEn,DisplayName
5692,22,527,True,1953-08-10,Prince Edward Island,6971,Queen's,10086,Elected,2
3,1,54,False,1871-03-02,Manitoba,5160,Marquette,282,Elected,2
333,2,226,True,1872-07-20,Ontario,6253,Ottawa (City of),0,Elected,2
442,2,480,False,1873-09-29,Prince Edward Island,4214,King's County,0,Elected,2
443,2,480,False,1873-09-29,Prince Edward Island,6782,Prince County,0,Elected,2
444,2,480,False,1873-09-29,Prince Edward Island,6978,Queen's County,0,Elected,2
641,3,390,True,1874-01-22,Prince Edward Island,6978,Queen's County,0,Elected,2
1772,7,288,False,1892-02-11,Nova Scotia,3479,Halifax,0,Elected,2
3146,13,383,True,1917-12-17,Nova Scotia,3479,Halifax,0,Elected,2


In [216]:
wins[(wins['ElectionId'] == 527) & (wins['ConstituencyId'] == 6971)]

,ParliamentNumber,ElectionId,IsGeneral,ElectionDate,ConstituencyId,ConstituencyEn,ProvinceEn,DisplayName,ElectionCandidateId,ElectionProcessCandidateId,...,ElectionCanadaLastName,ElectionCanadaMiddleName,Gender,OccupationEn,PartyOrganizationId,PartyNameEn,Votes,HasProfile,ResultLongEn,OtherResultLongEn
25334,22,527,True,1953-08-10,6971,Queen's,Prince Edward Island,"MacLean, John Angus",16837,0,...,MACLEAN,None,M,Farmer,6876,Progressive Conservative Party,10086,True,Elected,None
25504,22,527,True,1953-08-10,6971,Queen's,Prince Edward Island,"MacLean, John Angus",43267,0,...,Maclean,None,M,Farmer,6876,Progressive Conservative Party,10086,True,Elected,None
25337,22,527,True,1953-08-10,6971,Queen's,Prince Edward Island,"Matheson, Neil Alexander",17175,0,...,MATHESON,None,M,Editor,4831,Liberal Party of Canada,10351,True,Elected,None


In [217]:
candidates.to_csv('../data/parliaments/candidates.csv', index=False, encoding='utf8')

# Roles v2

In [ ]:
with open(os.path.join(people_dir, 'parlinfo_28-1_28-2_28-3.json')) as f:
    people = json.load(f)

In [100]:
person = people[0]
sorted(person.keys())

['Age',
 'AssistantCriticOfEn',
 'AssistantCriticOfFr',
 'AssistantDeputySpeakerChairEn',
 'AssistantDeputySpeakerChairFr',
 'AssociateMinisterOfEn',
 'AssociateMinisterOfFr',
 'Bibliography',
 'CabinetChairEn',
 'CabinetChairFr',
 'CabinetCommitteeMemberEn',
 'CabinetCommitteeMemberFr',
 'CityOfBirthEn',
 'CityOfBirthFr',
 'ConstituencyAddresses',
 'ConstituencyEn',
 'ConstituencyFr',
 'CountryOfBirthEn',
 'CountryOfBirthFr',
 'CriticOfEn',
 'CriticOfFr',
 'CurrentConstituencyEn',
 'CurrentConstituencyFr',
 'CurrentPartyEn',
 'CurrentPartyFr',
 'DateOfBirth',
 'DateOfBirthIsApproximate',
 'Death',
 'DeputyHouseLeaderEn',
 'DeputyHouseLeaderFr',
 'DeputyPrimeMinister',
 'DeputySpeaker',
 'DeputyWhipEn',
 'DeputyWhipFr',
 'DiedInOffice',
 'DisplayName',
 'DoNotDiscloseBirthLocation',
 'DoNotDiscloseDateOfBirth',
 'Education',
 'ElectionCandidates',
 'EthnicityLongEn',
 'EthnicityLongFr',
 'ExternalNotesEn',
 'ExternalNotesFr',
 'FamilyRelations',
 'FormalFirstName',
 'Gender',
 'HOCPers

In [101]:
[k for k, v in person.items() if type(v) in [list, dict]]

['Professions',
 'FamilyRelations',
 'Pictures',
 'Roles',
 'Death',
 'YearsOfServiceSegments']

In [ ]:
person_cols = ['PersonId', 'LastName', 'UsedFirstName']
rows = []
for rec in person_recs():
    person = rec['Person']
    person_props = {col: person[col] for col in person_cols}
    for role in person['Roles']:
        role_props = {k: v for k, v in role.items() if not (k.endswith('Fr') or type(v) in [list, dict])}
        row = {**person_props, **role_props}
        rows.append(row)
len(rows)

In [ ]:
sorted(rec.keys())

In [ ]:
sorted(rec['Person'].keys())

In [ ]:
rows[0]

In [ ]:
sorted(rows[0].keys())

In [103]:
df = pd.DataFrame(rows)
cleanup(df)
df.count()

PersonId                  202575
LastName                  202575
UsedFirstName             202575
PersonRoleId              202575
StartDate                 200744
StartDateIsApproximate    202575
EndDate                   198138
EndDateIsApproximate      202575
EndReasonEn                  230
EndReasonTypeEn             1685
SourceOfInformationEn       1139
IsActing                  202575
ActingTextEn                  11
HasCrossedTheFloor        202575
OrganizationId            202575
OrganizationLongEn        202575
OrganizationShortEn       196184
OrganizationAcronymEn     190713
OrganizationTypeId        202575
OrganizationTypeEn        202547
OrganizationProvinceEn      6958
OrganizationHasProfile    202575
IsSenatorialDivision      202575
ToBeStyledAsEn            180918
GroupId                   202575
Source                      8535
PortFolioEn                 7426
GroupingTitleEn            31193
GroupingOrder             202575
ParliamentStart            25827
Parliament

In [128]:
df.sample(25).sort_values(['LastName', 'UsedFirstName', 'StartDate'])

,PersonId,LastName,UsedFirstName,PersonRoleId,StartDate,StartDateIsApproximate,EndDate,EndDateIsApproximate,EndReasonEn,EndReasonTypeEn,...,PartyEn,PartyStartDate,PartyEndDate,IsMP,IsSenator,NotesEn,IsCurrent,RoleId,NameEn,Ordinal
1219,958,Aikins,James Albert Manning,30400,1911-11-15,False,1915-04-15,False,None,None,...,None,None,None,False,False,None,False,2998,Caucus Member,NaN
9011,15678,Baker,George,267854,1989-04-03,False,1991-05-12,False,None,None,...,Liberal Party of Canada,1972-10-30,2017-09-03,False,False,None,False,226,Member,300.0
17458,12709,Blain,Richard,226686,1903-03-12,False,1903-10-24,False,None,None,...,Conservative (1867-1942),1900-11-07,1926-11-27,False,False,None,False,226,Member,300.0
19360,14748,Borden,Frederick William,231209,1896-08-19,False,1896-10-05,False,None,None,...,Liberal Party of Canada,1896-07-30,1911-09-20,False,False,None,False,226,Member,300.0
26307,13311,Buchanan,William Ashbury,186153,1932-10-06,False,1933-05-27,False,None,None,...,Liberal Party of Canada,1925-09-05,1954-07-11,False,False,None,False,226,Member,300.0
33078,6895,Carter,Chesley William,140835,1966-01-18,False,1967-05-08,False,None,None,...,Liberal Party of Canada,1949-06-27,1977-07-28,False,False,None,False,226,Member,300.0
42605,4154,Copp,Arthur Bliss,211731,1915-02-04,False,1915-04-15,False,None,None,...,Liberal Party of Canada,1915-02-01,1917-12-16,False,False,None,False,226,Member,300.0
50679,6281,Denis,Azellus,281462,1976-10-12,False,1977-10-17,False,None,None,...,Liberal Party of Canada,1935-10-14,1991-09-04,False,False,None,False,226,Member,300.0
61317,12772,Fairweather,Robert Gordon Lee,19061,1974-02-27,False,1974-05-09,False,None,None,...,Progressive Conservative Party,1962-06-18,1977-08-31,False,False,None,False,226,Member,300.0
78364,11908,Griesbach,William Antrobus,183562,1943-01-28,False,1944-01-26,False,None,None,...,Conservative (1867-1942),1921-09-15,1945-01-21,False,False,None,False,226,Member,300.0


In [120]:
filename = os.path.join(people_dir, 'roles_with_provincial.csv')
df.to_csv(filename, index=False, encoding='utf8')

In [121]:
!wc $filename

  202594 2361918 58958014 ../data/people/roles_with_provincial.csv


In [123]:
zip_filename = filename + '.zip'
!rm $zip_filename
!zip $zip_filename $filename

rm: ../data/people/roles_with_provincial.csv.zip: No such file or directory
  adding: ../data/people/roles_with_provincial.csv (deflated 92%)


In [124]:
!wc $zipfilename

^C


In [126]:
!ls -al $zip_filename

-rw-r--r--  1 nedgar  staff  4930803  3 May 14:17 ../data/people/roles_with_provincial.csv.zip
